In [1]:
import sys
import os
import keras_tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import PReLU, Reshape, SimpleRNN, Dropout, Dense
from tensorflow.keras import Sequential, Input


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. P

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 19.8372 - val_loss: 16.6928
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.1533 - val_loss: 14.6605
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.3771 - val_loss: 12.9378
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.2030 - val_loss: 10.5495
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.2072 - val_loss: 7.2688
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5738 - val_loss: 3.8973
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5971 - val_loss: 1.7956
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2801 - val_loss: 1.1148
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9325 - val_loss: 0.9165
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7937 - val_loss: 0.8007
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6944 - val_loss: 0.7080
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - l

C:\Users\A_118784\Desktop\matura_github\matura\matura\FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


-5 + -1 - -3
2543
-3.0

Expressions not in x:
3 + 1 - 3
True
1457
1.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.0001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [4]:
input_shape = x_train[0].shape
from tensorflow.keras import backend as K

def build_model(hp, input_shape):
    K.clear_session()
    num_neurons = hp.Int("num_neurons", 32, 512) # Started range from 32 for efficiency
    num_layers = hp.Int("num_layers", 1, 5) # Reduced max layers for typical use cases
    dropoutTF = hp.Boolean("dropoutT/F")
    
    lil_model = Sequential()
    lil_model.add(Input(shape=input_shape))
    
    lil_model.add(Reshape((input_shape[0], 1)))

    for i in range(num_layers):
        is_last_rnn = (i == num_layers - 1)
        lil_model.add(SimpleRNN(num_neurons, return_sequences=not is_last_rnn))
        lil_model.add(PReLU())
        if dropoutTF:
            lil_model.add(Dropout(0.1)) # Using a common dropout rate

    lil_model.add(Dense(1, activation='linear'))
    lil_model.compile(optimizer="adam", loss="mse")
    return lil_model

In [5]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=20,
    executions_per_trial=1,
    overwrite=False,
    directory="RNNTuner",
    project_name="tuner",
)

Reloading Tuner from RNNTuner\tuner\tuner0.json


In [6]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

In [7]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 15, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 15, 269)             │          72,899 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ p_re_lu (PReLU)                      │ (None, 15, 269)             │           4,035 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 269)                 │         144,991 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ p_re_lu_1 (PReLU)                    │ (None, 269)                 │             269 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             270 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 222,464 (869.00 KB)

 Trainable params: 222,464 (869.00 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in RNNTuner\tuner
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 01 summary
Hyperparameters:
num_neurons: 269
num_layers: 2
dropoutT/F: False
Score: 0.04139498993754387

Trial 06 summary
Hyperparameters:
num_neurons: 105
num_layers: 1
dropoutT/F: False
Score: 0.056369248777627945

Trial 00 summary
Hyperparameters:
num_neurons: 157
num_layers: 4
dropoutT/F: False
Score: 0.05695940554141998

Trial 03 summary
Hyperparameters:
num_neurons: 260
num_layers: 3
dropoutT/F: False
Score: 0.06268113106489182

Trial 12 summary
Hyperparameters:
num_neurons: 32
num_layers: 3
dropoutT/F: False
Score: 0.07558765262365341

Trial 17 summary
Hyperparameters:
num_neurons: 234
num_layers: 1
dropoutT/F: True
Score: 0.08195668458938599

Trial 19 summary
Hyperparameters:
num_neurons: 228
num_layers: 5
dropoutT/F: False
Score: 0.08496366441249847

Trial 11 summary
Hyperparameters:
num_neurons: 32
num_layers: 5
dropoutT/F: False
Score: 0.09728255122900009

Tria

In [8]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [9]:
best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 16.9346 - val_loss: 16.1657
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 11.7702 - val_loss: 7.6825
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 5.9466 - val_loss: 2.7847
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2.3725 - val_loss: 3.8987
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2.5028 - val_loss: 1.0905
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.7227 - val_loss: 0.6065
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.4087 - val_loss: 0.3401
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2687 - val_loss: 0.2633
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.2398 - val_loss: 0.3802
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2384 - val_loss: 0.3018
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2222 - val_loss: 0.3180
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/s

In [10]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [11]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.2479258
MRE in Range:  0.10239994454049685
MAE longer Expressions:  5.3757056292944725
MAE out Range:  3.735671
MRE out Range:  0.36823938993991323


In [12]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [13]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [14]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

0.06614721051618623
0.09767732027118708
0.1042826194756733
0.06400072786257181
Benchmark: 0.3321078781256184


In [15]:
# New cell for RNN2 notebook - Statistical analysis with multiple runs
#this is also generated based on FNN6.py (DeepseekR1 this time.) (ofc, slightly tweaked as always)
# Initialize lists to store metrics across multiple runs
benchmarks_rnn = []
MAEinRange_rnn = []
MREinRange_rnn = []
MAEoutRange_rnn = []
MREoutRange_rnn = []
MAElongRange_rnn = []

# Run multiple training iterations
for progress in range(5):
    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.0001,
        restore_best_weights=True,
        monitor='val_loss',
        mode = "min"
    )
    print(f"Progress: {progress + 1}/5")
    
    # Build and train model
    current_model = build_model(best_hps, input_shape)
    
    current_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=200,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Make predictions
    predsInRange = current_model.predict(x_test_dataset, verbose=0)
    predsOutRange = current_model.predict(out_x_test_dataset, verbose=0)
    predsLongRange = current_model.predict(long_x_test_dataset, verbose=0)
    
    # Calculate metrics
    reldiffInRange = []
    diffInRange = []
    safe_y_test = np.where(np.isclose(y_test, 0.0), 1.0, y_test)
    
    for i in range(len(y_test)):
        diffInRange.append(abs(y_test[i] - predsInRange[i]))
        reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
    
    MAEinRange_rnn.append(np.mean(diffInRange))
    MREinRange_rnn.append(np.mean(reldiffInRange))

    # Long range metrics
    NEEDdiffLongRange = []
    for i in range(len(long_y_test)):
        NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    MAElongRange_rnn.append(np.mean(NEEDdiffLongRange))
    
    # Out of range metrics
    diffOutRange = []
    for i in range(len(out_y_test)):
        diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
    safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
    diff_out_relError = []
    for i in range(len(out_y_test)):
        diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
    
    MAEoutRange_rnn.append(np.mean(diffOutRange))
    MREoutRange_rnn.append(np.mean(diff_out_relError))
    
    # Calculate benchmark
    placeholder = absSum(outsideExpr)
    diffOutRange_22 = []
    indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 
    
    for i in indices_with_placeholder_22:
        diffOutRange_22.append(np.abs(out_y_test[i]-predsOutRange[i]))
    
    meanDiff_InRange = np.mean(diffInRange)
    meanDiff_OutRange = np.mean(diffOutRange_22)
    meanDiff_LongRange = np.mean(NEEDdiffLongRange)
    meanDiff_OutRelRange = np.mean(diff_out_relError)
    
    benchmark = 0
    benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
    benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
    benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
    benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
    
    benchmarks_rnn.append(benchmark)

# Statistical analysis
from scipy.stats import ttest_1samp


stats6, p_value6 = ttest_1samp(benchmarks_rnn, popmean=1)

print(f"RNN Model - Statistical Analysis Results:")

print(f"Benchmark P-value: {p_value6:.6f}")

print(f"\nRNN Model - Average Metrics:")
print(f"Average MAE in Range: {np.mean(MAEinRange_rnn):.6f}")
print(f"Average MRE in Range: {np.mean(MREinRange_rnn):.6f}")
print(f"Average MAE out Range: {np.mean(MAEoutRange_rnn):.6f}")
print(f"Average MRE out Range: {np.mean(MREoutRange_rnn):.6f}")
print(f"Average MAE long Range: {np.mean(MAElongRange_rnn):.6f}")
print(f"Average benchmark: {np.mean(benchmarks_rnn):.6f}")

print(f"\nRNN Model - All Runs:")
print(f"MAE in Range: {[f'{x:.6f}' for x in MAEinRange_rnn]}")
print(f"MRE in Range: {[f'{x:.6f}' for x in MREinRange_rnn]}")
print(f"MAE out Range: {[f'{x:.6f}' for x in MAEoutRange_rnn]}")
print(f"MRE out Range: {[f'{x:.6f}' for x in MREoutRange_rnn]}")
print(f"MAE long Range: {[f'{x:.6f}' for x in MAElongRange_rnn]}")
print(f"Benchmark: {[f'{x:.6f}' for x in benchmarks_rnn]}")

Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5
RNN Model - Statistical Analysis Results:
Benchmark P-value: 0.000006

RNN Model - Average Metrics:
Average MAE in Range: 0.241331
Average MRE in Range: 0.093253
Average MAE out Range: 3.874989
Average MRE out Range: 0.391931
Average MAE long Range: 5.457189
Average benchmark: 0.314651

RNN Model - All Runs:
MAE in Range: ['0.216918', '0.276503', '0.189650', '0.298225', '0.225356']
MRE in Range: ['0.084556', '0.109808', '0.071775', '0.113635', '0.086494']
MAE out Range: ['3.988119', '3.959281', '3.480594', '4.013595', '3.933357']
MRE out Range: ['0.416637', '0.397483', '0.342886', '0.412531', '0.390119']
MAE long Range: ['5.265619', '5.629467', '5.309284', '5.528005', '5.553570']
Benchmark: ['0.317452', '0.280492', '0.392283', '0.271497', '0.311533']
